In [1]:
import pycocotools.coco as coco
import json
import os
import os.path as osp
from PIL import Image
from tqdm import tqdm
import numpy as np
import shutil
import cv2
import yaml

from PIL import Image 
import time

def to_pil(cv_img):
    img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img)

In [2]:
src = "/media/allen/mass"
train_label_path = osp.join(src, "annotation_train.odgt")
train_img_src = osp.join(src, "DB/crowdhuman/train2017")
val_label_path = osp.join(src, "annotation_val.odgt")
val_img_src = osp.join(src, "DB/crowdhuman/val2017")

In [3]:
dataset = {
    "info": {},
    "licenses": [],
    "images": [],
    "annotations": [],
    "categories": []
}
dataset['categories'].append({
    'id': 1,
    'name': "person",
    'supercategory': "people",
    'keypoints': [],
    'skeleton': []
})

In [5]:
sub_index = 0
idx = 0
with open(train_label_path, 'r') as f:
    for line in tqdm(f):
        anns = yaml.load(line)
        fname = osp.join(train_img_src, f"{anns['ID']}.jpg")
        img = Image.open(fname)
        img_w, img_h = img.size
        idx += 1
        dataset['images'].append({
            'coco_url': '',
            'date_captured': '',
            'file_name': osp.basename(fname),
            'flickr_url': '',
            'id': idx,
            'license': 0,
            'width': img_w,
            'height': img_h
        })
        for bbox in anns['gtboxes']:
            if 'extra' in bbox and 'ignore' in bbox['extra'] and bbox['extra']['ignore'] == 1:
                continue
            if 'extra' in bbox and 'unsure' in bbox['extra'] and bbox['extra']['unsure'] == 1:
                continue 
            sub_index += 1
            x, y, w, h = bbox['fbox']
            if h/w < 1.2:
                continue
            dataset['annotations'].append({
                'area': w*h,
                'bbox': [max(x,0), max(y,0), w, h],
                'category_id': 1,
                'id': sub_index,
                'image_id': idx,
                'iscrowd': 0,
                'segmentation': [],
            })

15000it [22:16, 12.74it/s]


In [42]:
img = cv2.imread(osp.join(train_img_src, f"{labels[2]['ID']}.jpg"))
anns = labels[2]
for bbox in anns['gtboxes']:
    x, y, w, h = bbox['fbox']
    cv2.rectangle(img, (max(x, 0), max(y, 0)), (max(x+w, 0), max(y+h, 0)), (0,255,0), 2)

In [6]:
label_path = osp.join(src, "DB/crowdhuman/original")
with open(osp.join(label_path, 'instances_train2017.json'), 'w') as f:
    json.dump(dataset, f)

In [4]:
sub_index = 0
idx = 0
with open(val_label_path, 'r') as f:
    for line in tqdm(f):
        anns = yaml.load(line)
        fname = osp.join(val_img_src, f"{anns['ID']}.jpg")
        img = Image.open(fname)
        img_w, img_h = img.size
        idx += 1
        dataset['images'].append({
            'coco_url': '',
            'date_captured': '',
            'file_name': osp.basename(fname),
            'flickr_url': '',
            'id': idx,
            'license': 0,
            'width': img_w,
            'height': img_h
        })
        for bbox in anns['gtboxes']:
            if 'extra' in bbox and 'ignore' in bbox['extra'] and bbox['extra']['ignore'] == 1:
                continue
            if 'extra' in bbox and 'unsure' in bbox['extra'] and bbox['extra']['unsure'] == 1:
                continue 
            sub_index += 1
            x, y, w, h = bbox['fbox']
            if h/w < 1.2:
                continue
            dataset['annotations'].append({
                'area': w*h,
                'bbox': [max(x,0), max(y,0), w, h],
                'category_id': 1,
                'id': sub_index,
                'image_id': idx,
                'iscrowd': 0,
                'segmentation': [],
            })

4370it [05:05, 14.29it/s]


In [5]:
label_path = osp.join(src, "DB/crowdhuman/original")
with open(osp.join(label_path, 'instances_val2017.json'), 'w') as f:
    json.dump(dataset, f)

In [6]:
co = coco.COCO(osp.join(label_path, 'instances_val2017.json'))

loading annotations into memory...
Done (t=0.65s)
creating index...
index created!


In [10]:
image_ids = co.getImgIds()
img_id = image_ids[139]
idxs = co.getAnnIds(imgIds=[img_id])
fname = co.loadImgs(ids=[img_id])[0]['file_name']

In [11]:
img = Image.open(os.path.join(val_img_src, fname))
anns = co.loadAnns(idxs)

In [13]:
np_img = np.array(img)
for ann in anns:
    x1, y1, w, h = ann['bbox']
    cv2.rectangle(np_img, (int(x1), int(y1)), (int(x1+w), int(y1+h)), (255,255,255), 2)
Image.fromarray(np_img))

SyntaxError: invalid syntax (<ipython-input-13-38b985e264d0>, line 5)